In [1]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras import layers, models
import cv2
import os
import random 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# importinng libarys

In [3]:
class RextRecoginitionmodel:
    def __init__(self, input_shape=(128,128,1), learning_rate=0.001):
        self.input_shape = input_shape
        self.learning_rate =  learning_rate
        self.model = None
        self.history =None
        self.characters = None 

    def build_model(self, num_classes):

        model = models.Sequential([
            # first convolution layer 
            layers.Conv2D(32,(3,3), activation ='relu', padding ='same', input_shape =self.input_shape),
            layers.MaxPooling2D((2,2)),
            layers.BatchNormalization(),

            # second convolutation layer
            layers.Conv2D(64,(3,3), activation='relu', padding= 'same', input_shape= self.input_shape),
            layers.MaxPooling2D((2,2)),
            layers.BatchNormalization(),
        # third convolution layer 
            layers.Conv2D(128,(3,3), activation='relu', padding='same'),
            layers.MaxPooling2D((2,2)),
            layers.BatchNormalization(),
        # forth convolutation layer
            layers.Conv2D(256,(3,3), activation='relu', padding='same'),
            layers.MaxPooling2D((2,2)),
            layers.BatchNormalization(),
        # flatten and dense layer 
            layers.Flatten(),
            layers.Dense(256, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation='softmax')
            
            
        ])
        

In [ ]:
# compile the model 
    model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
    loss = 'sparse_categorical_crossentropy',
    meterics = ['accuracy']
    )
    self.model= model 
    return model 
def preprocess_image(self, image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Could not  read image at{image_path}")

    image = cv2.resize(image, (self.input_shape[1], self.input_shape[0]))

    image = image/255.0

    image  = np.expand_dims(image, axis=-1)
    return image

def prepare_dataset(self, image_paths, labels):
    unique_chars = set()
    for label in labels:
        unique_chars.update(list(label))
    self.characeters = sorted(list(unique_chars))
    char_to_idx = {char: idx for idx, char in enumerate(self.characeters)}

    images = []
    indices = []:
    
    for image_path, label in zip(image_paths, labels):
        try:
            image = self.preprocess_image(image_path)
            images.append(image)

            indices.append(char_to_idx[label[0]])
        except Exception as e:
            print(f"error processing{image_path}: {e}")

    X = np.array(images)
    y = np.array(indices)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

def train(self, image_paths, labels, epochs=10, batch_size=32):
    X_train, X_val, y_train, y_val = self.prepare_dataset(image_paths, labels)

    if self.model is None:
        self.build_model(len(self.characeters))

    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizonatal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.1),
    ])

    # train model 
    self.history = self.model.fit(
        X_train, y_train,
        epochs= epochs,
        batch_size= batch_size,
        validation_data = (X_val, y_val),
        callback = [
            tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights= True),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=2)
        ]
    )
    return self.history
def save_model(self, path):
    if self.model is None:
        raise ValueError("No model has been trained yet")

        np.save(os.path.join.dirname(path),"characters.np")